In [9]:
import sys
import jsonlines
import os
import pprint
import time
import random
import re
import json
import math

pp = pprint.PrettyPrinter()
sys.path.insert(0, "/home/martin/FEVER2021_SharedTask/FEVEROUS/src")

# DIR_PATH = "e:\\Documents\\NLP\\FEVER2021_SharedTask\\"
TRAIN_DATA_PATH = "train_data/train.jsonl"

from database.feverous_db import FeverousDB
from utils.wiki_page import WikiPage

### Import data

In [11]:
db = FeverousDB("/media/martin/Windows/data/feverous_wikiv1.db")

#### Training data

In [2]:
train_data = []
with jsonlines.open(TRAIN_DATA_PATH) as reader:
    for i, doc in enumerate(reader):
        train_data.append(doc)

In [4]:
claim_text = "United States sanctions includes the countries of China, Lesotho, Philippines"
train_sample = [data for data in train_data if claim_text in data["claim"]][0]
train_sample

{'evidence': [{'content': ['Trafficking in Persons Report_cell_0_137_0',
    'Trafficking in Persons Report_cell_0_137_11',
    'Trafficking in Persons Report_cell_0_151_0',
    'Trafficking in Persons Report_cell_0_151_11'],
   'context': {'Trafficking in Persons Report_cell_0_137_0': ['Trafficking in Persons Report_title',
     'Trafficking in Persons Report_section_0',
     'Trafficking in Persons Report_header_cell_0_0_0'],
    'Trafficking in Persons Report_cell_0_137_11': ['Trafficking in Persons Report_title',
     'Trafficking in Persons Report_section_0',
     'Trafficking in Persons Report_header_cell_0_0_11'],
    'Trafficking in Persons Report_cell_0_151_0': ['Trafficking in Persons Report_title',
     'Trafficking in Persons Report_section_0',
     'Trafficking in Persons Report_header_cell_0_0_0'],
    'Trafficking in Persons Report_cell_0_151_11': ['Trafficking in Persons Report_title',
     'Trafficking in Persons Report_section_0',
     'Trafficking in Persons Report_h

In [24]:
train_sample = [data for data in train_data if data["id"] == 68924][0]
train_sample

{'evidence': [{'content': ['List of Chicago Blackhawks head coaches_cell_1_23_1',
    'List of Chicago Blackhawks head coaches_cell_1_23_2',
    'List of Chicago Blackhawks head coaches_cell_1_28_2',
    'List of Chicago Blackhawks head coaches_cell_1_28_1'],
   'context': {'List of Chicago Blackhawks head coaches_cell_1_23_1': ['List of Chicago Blackhawks head coaches_title',
     'List of Chicago Blackhawks head coaches_section_1',
     'List of Chicago Blackhawks head coaches_header_cell_1_0_1'],
    'List of Chicago Blackhawks head coaches_cell_1_23_2': ['List of Chicago Blackhawks head coaches_title',
     'List of Chicago Blackhawks head coaches_section_1',
     'List of Chicago Blackhawks head coaches_header_cell_1_0_2'],
    'List of Chicago Blackhawks head coaches_cell_1_28_2': ['List of Chicago Blackhawks head coaches_title',
     'List of Chicago Blackhawks head coaches_section_1',
     'List of Chicago Blackhawks head coaches_header_cell_1_0_2'],
    'List of Chicago Blackh

In [25]:
PAGE_NAME = "List of Chicago Blackhawks head coaches"
page_json = db.get_doc_json(PAGE_NAME)
wiki_page = WikiPage(PAGE_NAME, page_json)
tables = wiki_page.get_tables()
table_rows = tables[1].get_rows()
rows = [table_row.cell_content for table_row in table_rows]
rows

[['#',
  'Name',
  'Term',
  'Regular season',
  'Regular season',
  'Regular season',
  'Regular season',
  'Regular season',
  'Regular season',
  'Playoffs',
  'Playoffs',
  'Playoffs',
  'Playoffs',
  'Playoffs',
  'Achievements',
  'Ref'],
 ['#',
  'Name',
  'Term',
  'GC',
  'W',
  'L',
  'T/OT',
  'PTS',
  'Win%',
  'GC',
  'W',
  'L',
  'T',
  'Win%',
  'Achievements',
  'Ref'],
 ['1',
  '[[Pete_Muldoon|Pete Muldoon]]*',
  '[[1926–27_NHL_season|1926–1927]]',
  '44',
  '19',
  '22',
  '3',
  '41',
  '.466',
  '2',
  '0',
  '1',
  '1',
  '.250',
  '',
  ''],
 ['2',
  '[[Barney_Stanley|Barney Stanley]]*',
  '[[1927–28_NHL_season|1927–1928]]',
  '23',
  '4',
  '17',
  '2',
  '10',
  '.217',
  '—',
  '—',
  '—',
  '—',
  '—',
  '',
  ''],
 ['3',
  '[[Hughie_Lehman|Hughie Lehman]]*',
  '[[1927–28_NHL_season|1928]]',
  '21',
  '3',
  '17',
  '1',
  '7',
  '.167',
  '—',
  '—',
  '—',
  '—',
  '—',
  '',
  ''],
 ['4',
  '[[Herb_Gardiner|Herb Gardiner]]*',
  '[[1928–29_NHL_season|1928–1

In [28]:
row_lens = [len(row) for row in rows]
row_lens

[16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16]

In [43]:
import ast
import pandas as pd
tapas_table = pd.read_csv("data/tapas/tables/table_4239.csv").astype(str)

tapas_data = pd.read_csv("data/tapas/train_data/tapas_data.csv", converters={
    "answer_coordinates": ast.literal_eval,
    "answer_text": ast.literal_eval
})
idx = 1524
item = tapas_data.iloc[idx]
answer_texts = tapas_data["answer_text"][idx]
for i, coords in enumerate(item.answer_coordinates):
    answer_table = tapas_table.iloc[coords[0], coords[1]+1]
    answer_text = answer_texts[i]
    if answer_table != answer_text:
        print("answer table: {}".format(answer_table))
        print("answer text: {}".format(answer_text))
    assert answer_table == answer_text

In [37]:
tapas_data["answer_text"][1524]

['Tommy Ivan†', '1956–1957', '1979–1980', 'Eddie Johnston']

In [42]:
tapas_table.iloc[23, 2]

'Tommy Ivan†'

In [41]:
tapas_table

,Unnamed: 0,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15
0,0,#,Name,Term,Regular season,Regular season,Regular season,Regular season,Regular season,Regular season,Playoffs,Playoffs,Playoffs,Playoffs,Playoffs,Achievements,Ref
1,1,#,Name,Term,GC,W,L,T/OT,PTS,Win%,GC,W,L,T,Win%,Achievements,Ref
2,2,1,Pete Muldoon*,1926–1927,44,19,22,3,41,.466,2,0,1,1,.250,nan,nan
3,3,2,Barney Stanley*,1927–1928,23,4,17,2,10,.217,—,—,—,—,—,nan,nan
4,4,3,Hughie Lehman*,1928,21,3,17,1,7,.167,—,—,—,—,—,nan,nan
5,5,4,Herb Gardiner*,1928–1929,32,5,23,4,14,.219,—,—,—,—,—,nan,nan
6,6,5,Dick Irvin,1929,12,2,6,4,8,.333,—,—,—,—,—,nan,nan
7,7,6,Tom Shaughnessy*,1929–1930,21,10,8,3,23,.548,—,—,—,—,—,nan,nan
8,8,7,Bill Tobin*,1930,23,11,10,2,24,.522,2,0,1,1,.250,nan,nan
9,9,—,Dick Irvin,1930–1931,44,24,17,3,51,.580,9,5,3,1,.611,nan,nan


In [27]:
tables[1].get_ids()

['header_cell_1_0_0',
 'header_cell_1_0_1',
 'header_cell_1_0_2',
 'header_cell_1_0_3',
 'header_cell_1_0_3',
 'header_cell_1_0_3',
 'header_cell_1_0_3',
 'header_cell_1_0_3',
 'header_cell_1_0_3',
 'header_cell_1_0_9',
 'header_cell_1_0_9',
 'header_cell_1_0_9',
 'header_cell_1_0_9',
 'header_cell_1_0_9',
 'header_cell_1_0_14',
 'header_cell_1_0_15',
 'header_cell_1_0_0',
 'header_cell_1_0_1',
 'header_cell_1_0_2',
 'header_cell_1_1_0',
 'header_cell_1_1_1',
 'header_cell_1_1_2',
 'header_cell_1_1_3',
 'header_cell_1_1_4',
 'header_cell_1_1_5',
 'header_cell_1_1_6',
 'header_cell_1_1_7',
 'header_cell_1_1_8',
 'header_cell_1_1_9',
 'header_cell_1_1_10',
 'header_cell_1_0_14',
 'header_cell_1_0_15',
 'cell_1_2_0',
 'cell_1_2_1',
 'cell_1_2_2',
 'cell_1_2_3',
 'cell_1_2_4',
 'cell_1_2_5',
 'cell_1_2_6',
 'cell_1_2_7',
 'cell_1_2_8',
 'cell_1_2_9',
 'cell_1_2_10',
 'cell_1_2_11',
 'cell_1_2_12',
 'cell_1_2_13',
 'cell_1_2_14',
 'cell_1_2_15',
 'cell_1_3_0',
 'cell_1_3_1',
 'cell_1_3_2',


In [ ]:
train_example = train_data[0]
evidence = train_example['evidence']
PAGE_NAME = "Tammy Garcia"
page_json = db.get_doc_json(PAGE_NAME)
wiki_page = WikiPage(PAGE_NAME, page_json)

def get_sent_evidence(train_json):    
    for e in evidence:
        sent_ids = []
        content = e['content']
        for c in content:
            sent_id = c.replace(PAGE_NAME + "_", "")
            sent_ids.append(sent_id)

    sentences = wiki_page.get_sentences()
    content_text = ""
    for sent_id in sent_ids:
        for sent in sentences:
            if sent.name == sent_id:
                content_text += sent.content + " "
                break

print(content_text)

In [ ]:
start_time = time.time()
doc_ids = db.get_doc_ids()
print("Nr of docs: {} took {} seconds to fetch".format(len(doc_ids), time.time()-start_time))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from glob import glob
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import numpy as np

CORPUS_PATH = DIR_PATH + 'data\\corpora\\'

porter_stemmer = PorterStemmer()
s_words = set(stopwords.words('english'))

def create_corpus(max_files):
    file_paths = glob(CORPUS_PATH + '*.json')
    for i, f_path in enumerate(file_paths):
        if i > max_files:
            return
        print("Opening file '{}'".format(f_path))
        with open(f_path, 'r') as f:
            docs = json.loads(f.read())
            for key in docs:
                yield docs[key]

def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [word for word in words if word not in s_words]
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [ ]:
from multiprocessing import Pool

with open(CORPUS_PATH + 'corpora_1.json', 'r') as f:
    file_json = json.loads(f.read())
    test_strs = list(file_json.values())
start_time = time.time()
TEST_SIZE = 1000

# with Pool() as pool:
  #  pool.map(stemming_tokenizer, test_strs[:TEST_SIZE])
for test_str in test_strs[:TEST_SIZE]:
    stemmed_words = stemming_tokenizer(test_str)
print("Stemming {} docs {} seconds".format(TEST_SIZE, time.time() - start_time))
pp.pprint(stemmed_words)

In [13]:
len(file_json)

100000

In [ ]:
start_time = time.time()
# Without stemming
# tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words='english',dtype=np.float32)
# With stemming
tfidfvectorizer = TfidfVectorizer(stop_words='english', tokenizer=stemming_tokenizer)
corpus = create_corpus(5)
tfidf_wm = tfidfvectorizer.fit_transform(corpus)
print("Creating TF-IDF matrix took {} seconds".format(time.time() - start_time))

In [ ]:
from glob import glob
CORPUS_PATH = DIR_PATH + 'data\\corpora\\'

def create_doc_id_map():
    doc_id_map = []
    file_paths = glob(CORPUS_PATH + '*.json')
    for f_path in file_paths:
        with open(f_path, 'r') as f:
            docs = json.loads(f.read())
            for key in docs:
                doc_id_map.append(key)
    return doc_id_map

doc_id_map = create_doc_id_map()

In [ ]:
import pickle
pickle.dump(tfidfvectorizer, open("vectorizer-32bit.pickle", "wb"))
pickle.dump(tfidf_wm, open("tfidf_wm-32bit.pickle", "wb"))

In [3]:
import pickle

tfidfvectorizer = pickle.load(open("models\\vectorizer-32bit.pickle", "rb"))
tfidf_wm = pickle.load(open("models\\tfidf_wm-32bit.pickle", "rb"))

In [ ]:
with open(DIR_PATH + 'data\\doc_id_map.json', 'r') as f:
    doc_id_map = json.loads(f.read())

In [4]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100)
svd_matrix = svd.fit_transform(tfidf_wm)
svd_matrix.shape

(5421406, 100)

In [11]:
query_tfidf.shape

(1, 5897086)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

NR_TEST_QUERIES = 1000
test_queries = [train_data[i]['claim'] for i in range(NR_TEST_QUERIES)]

# test_query = train_data[0]['claim']
query_tfidf = tfidfvectorizer.transform(test_queries)
svd_query = svd.fit_transform(query_tfidf)
cosine_similarities = cosine_similarity(svd_query, svd_matrix).flatten()
related_docs_indices = cosine_similarities.argsort()[:-6:-1]
related_docs_indices

In [ ]:
cosine_similarities[:-6:-1]
# cosine_similarities.sort()[:-6:-1]

In [ ]:
doc_id_map[4244298]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import trange

def get_top_docs(claim):
    query_tfidf = tfidfvectorizer.transform([claim])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_wm).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-6:-1]
    return [doc_id_map[i] for i in related_docs_indices]

TEST_SAMPLE = 10
claim_top_docs = []
for i in trange(TEST_SAMPLE):
    claim = train_data[i]['claim']
    claim_top_docs.append(get_top_docs(claim))
    
print(claim_top_docs)

In [ ]:
pp.pprint(claim_top_docs)

In [ ]:
train_data[1]['claim']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def get_top_sents(doc_id, claim):
    doc_json = db.get_doc_json(doc_id)
    page = WikiPage(doc_json['title'], doc_json)
    sents = extract_sents(doc_json)
    sent_vectorizer = TfidfVectorizer(analyzer='word',stop_words='english')
    sent_wm = sent_vectorizer.fit_transform(sents)
    claim_tfidf = sent_vectorizer.transform([claim])
    cosine_similarities = cosine_similarity(claim_tfidf, sent_wm).flatten()
    top_sents_indices = cosine_similarities.argsort()[:-6:-1]
    print(top_sents_indices)
    return [sent for i, sent in enumerate(sents) if i in top_sents_indices]
    
claim = train_data[0]['claim']
doc_id = 'Tammy Garcia'
print(get_top_sents(doc_id, claim))

In [ ]:
claim

In [ ]:
doc_json = db.get_doc_json(doc_id)
pp.pprint(doc_json)

In [ ]:
len(train_data)